In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 8.5MB/s 
     |████████████████████████████████| 460kB 53.7MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [1]:
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

from konlpy.tag import Komoran
from konlpy.tag import Okt

Seq2Seq에서의 임베딩이 이전 예제와 다른 점은 아래와 같이 태그를 사용한다는 것입니다. 임베딩의 0~3번째에 각각 PADDING, START, END, OOV 태그를 넣습니다. 사실 그냥 똑같은 단어라고 보시면 됩니다. 다만 이 단어들이 Seq2Seq의 동작을 제어합니다. 

예를 들어, 디코더 입력에 START가 들어가면 디코딩의 시작을 의미합니다. 반대로 디코더 출력에 END가 나오면 디코딩을 종료합니다.
<br>
<br>
<br>

In [2]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 100

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
wear_data = pd.read_csv("/content/drive/My Drive/자연어처리,추천/NLPer/chatbot_pra/csv_file/wear_MAIN_edited.csv")
print(wear_data.shape)
customer = wear_data[wear_data.SPEAKER == "고객"].SENTENCE
store = wear_data[wear_data.SPEAKER == "점원"].SENTENCE
intent = wear_data['MAIN']
category = wear_data['CATEGORY']
print(customer.shape, store.shape, intent.shape) # 질문의 개수와 답의 개수가 일치하지 않는다.

(15826, 20)
(8381,) (7445,) (15826,)


In [4]:
wear_data['CATEGORY_MAIN'] = wear_data[['CATEGORY', 'MAIN']].apply(' '.join, axis=1)

In [5]:
wear_data['CATEGORY_MAIN_SENTENCE'] = wear_data[['CATEGORY_MAIN', 'SENTENCE']].apply(' '.join, axis=1)

In [10]:
wear_data['CATEGORY_MAIN_SENTENCE']

0                신발 종류별신발제품문의요청 신발은 여기 있는 게 다예요?
1                 신발 종류별신발제품문의요청 네 성인이나 아동 다 있어요
2                   신발 종류별신발제품문의요청 발 사이즈 몇 신으세요?
3                           신발 종류별신발제품문의요청 230이요
4                     신발 착화감 편하게 신을 수 있는 거 찾으세요?
                          ...                   
15821                  액세서리 쿠폰시적립요청 이 쿠폰에 적립해주세요
15822     액세서리 쿠폰적용사음품문의 사은품 받으려면 쿠폰 몇 개 모아야 해요?
15823    액세서리 쿠폰적용사음품문의 쿠폰 몇 개 모으면 사은품 받을 수 있어요?
15824    액세서리 쿠폰적용사음품문의 쿠폰 몇 장 모아야 사은품 받을 수 있나요?
15825     액세서리 쿠폰적용사음품문의 사은품 받으려면 쿠폰 몇 장 있어야 돼요?
Name: CATEGORY_MAIN_SENTENCE, Length: 15826, dtype: object

In [6]:
customer = wear_data[wear_data.SPEAKER == "고객"].CATEGORY_MAIN_SENTENCE
store = wear_data[wear_data.SPEAKER == "점원"].CATEGORY_MAIN_SENTENCE

In [7]:
prev = "고객"
store_arr = []
customer_arr = []
store_stc = ""
customer_stc = ""

for i in range(wear_data.shape[0]):
    if (prev == wear_data.iloc[i].SPEAKER):
        if prev == "점원":
             store_stc += (" "+wear_data.iloc[i].CATEGORY_MAIN_SENTENCE)
        else : 
            customer_stc += (" "+wear_data.iloc[i].CATEGORY_MAIN_SENTENCE)
            
    elif prev == "점원":
        store_arr.append(store_stc)
        customer_stc = wear_data.iloc[i].CATEGORY_MAIN_SENTENCE
        prev = "고객"
    else :
        customer_arr.append(customer_stc)
        store_stc = wear_data.iloc[i].CATEGORY_MAIN_SENTENCE
        prev = "점원"

print(len(store_arr))
print(len(customer_arr))
print(store_arr[-1])
print(customer_arr[-1]) # 자료 상에서 이후에는 계속 고객의 물음만 계속된다. 코드 레벨에서 이 부분은 빼게 구현했다. (stc는 만들어지지만 arr에 append 안하게 된다.)

7301
7301
의류 인기제품문의 요즘 파스텔 톤이 유행이에요
의류 인기제품문의 요즘 유행하는 색깔이 뭐예요?


In [19]:
cat_main_Q = []
cat_main_A = []

sentence_Q = []
sentence_A = []

for Q in customer_arr:
    sentence = Q.split()
    cat_main_Q.append(' '.join(sentence[:2]))
    sentence_Q.append(' '.join(sentence[2:]))

for A in store_arr:
    sentence = A.split()
    cat_main_A.append(' '.join(sentence[:2]))
    sentence_A.append(' '.join(sentence[2:]))

question = []
answer = []

for Q in sentence_Q:
    question.append(Q.replace("[^\w]", " "))

for A in sentence_A:
    answer.append(A.replace("[^\w]", " "))

len(question), len(answer)

(7301, 7301)

<br>
<br>
# 단어 사전 생성

In [20]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        pair = tagger.pos(sentence)
        
        postag = ''
        for t in pair:
            postag += (t[0] + '/' + t[1] + ' ')
        sentences_pos.append(postag)
    
    return sentences_pos

In [21]:
# 형태소분석 수행
question = pos_tag(question)
answer = pos_tag(answer)

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(5):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q : 신발/Noun 은/Josa 여기/Noun 있는/Adjective 게/Noun 다예/Noun 요/Josa 
A : 네/Noun 성인/Noun 이나/Josa 아동/Noun 다/Adverb 있어요/Adjective 신발/Noun 종류/Noun 별/Modifier 신발/Noun 제품/Noun 문의/Noun 요청/Noun 발/Noun 사이즈/Noun 몇/Noun 신으세요/Verb 

Q : 230/Number 이요/Noun 
A : 편하게/Adjective 신/Noun 을/Josa 수/Noun 있는/Adjective 거/Noun 찾으세요/Verb 

Q : 네/Noun 봄/Noun 이니까/Josa 편하게/Adjective 신/Noun 을/Josa 수/Noun 있는/Adjective 거/Noun 
A : 이런/Adjective 건/Noun 어떠세요/Adjective 이런/Adjective 거도/Noun 신발/Noun 무척/Noun 편하거든요/Adjective 

Q : 굽/Noun 좀/Noun 높은/Adjective 거/Noun 없나요/Adjective 
A : 봄/Noun 상품/Noun 은/Josa 아직/Adverb 어른/Noun 제품/Noun 이/Josa 많이/Adverb 안/VerbPrefix 나왔습니다/Verb 

Q : 언제/Noun 들어와요/Verb 
A : 이번/Noun 주/Noun 지나면/Verb 들어올/Verb 거/Noun 예요/Josa 



In [27]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)
sentences.extend(cat_main_Q)
sentences.extend(cat_main_A)
words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

In [28]:
print("손님과 점원의 말에서 사용된 총 단어의 수 :",len(words))

손님과 점원의 말에서 사용된 총 단어의 수 : 6993


In [29]:
# 단어 출력
words[:20]


['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '나라/Noun',
 '올라오는/Verb',
 '추가/Noun',
 '지원/Noun',
 '책정/Noun',
 '하셨나요/Verb',
 '오만오천원/Noun',
 '해주는/Verb',
 '쓰시/Noun',
 '한텐/Noun',
 '호가/Noun',
 '차이는/Verb',
 '조금씩/Adverb',
 '프로포즈/Noun',
 '그런/Adjective',
 '장가/Noun']

In [30]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [31]:
word_to_index

{'<PADDING>': 0,
 '<START>': 1,
 '<END>': 2,
 '<OOV>': 3,
 '나라/Noun': 4,
 '올라오는/Verb': 5,
 '추가/Noun': 6,
 '지원/Noun': 7,
 '책정/Noun': 8,
 '하셨나요/Verb': 9,
 '오만오천원/Noun': 10,
 '해주는/Verb': 11,
 '쓰시/Noun': 12,
 '한텐/Noun': 13,
 '호가/Noun': 14,
 '차이는/Verb': 15,
 '조금씩/Adverb': 16,
 '프로포즈/Noun': 17,
 '그런/Adjective': 18,
 '장가/Noun': 19,
 '드라이크리닝여부문의': 20,
 '사가/Noun': 21,
 '편해야/Adjective': 22,
 '15000원/Number': 23,
 '실버/Noun': 24,
 '화장실/Noun': 25,
 '내려왔으면/Verb': 26,
 '높은거는/Adjective': 27,
 '했나/Verb': 28,
 '거울/Noun': 29,
 '5/Number': 30,
 '모르겠는데/Verb': 31,
 '독일/Noun': 32,
 '아예/Noun': 33,
 '틀립니다/Verb': 34,
 '안되겠죠/Adjective': 35,
 '가지네요/Verb': 36,
 '좋아하시는/Adjective': 37,
 '6천원/Number': 38,
 '액세서리디자인선택': 39,
 '봐주세요/Verb': 40,
 '돌잔치/Noun': 41,
 '해야/Verb': 42,
 '안되는데/Adjective': 43,
 '44만/Number': 44,
 '두꺼운거요/Adjective': 45,
 'in/Alpha': 46,
 '색상/Noun': 47,
 '선호/Noun': 48,
 '팔렸습니다/Verb': 49,
 '별/Modifier': 50,
 '없었어요/Adjective': 51,
 '25000원/Number': 52,
 '아홉시/Noun': 53,
 '놀러/Verb': 54,
 '보고있는데요/Verb': 5

In [32]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

In [37]:
cat_main_question = []
for idx in range(len(question)):
    cat_main_question.append(cat_main_Q[idx] + ' ' + question[idx])

cat_main_answer = []
for idx in range(len(answer)):
    cat_main_answer.append(cat_main_A[idx] + ' ' + answer[idx])

In [38]:
print(cat_main_question[:5])
print(cat_main_answer[:5])

['신발 종류별신발제품문의요청 신발/Noun 은/Josa 여기/Noun 있는/Adjective 게/Noun 다예/Noun 요/Josa ', '신발 종류별신발제품문의요청 230/Number 이요/Noun ', '신발 착화감 네/Noun 봄/Noun 이니까/Josa 편하게/Adjective 신/Noun 을/Josa 수/Noun 있는/Adjective 거/Noun ', '신발 굽높이문의 굽/Noun 좀/Noun 높은/Adjective 거/Noun 없나요/Adjective ', '신발 재입고문의 언제/Noun 들어와요/Verb ']
['신발 종류별신발제품문의요청 네/Noun 성인/Noun 이나/Josa 아동/Noun 다/Adverb 있어요/Adjective 신발/Noun 종류/Noun 별/Modifier 신발/Noun 제품/Noun 문의/Noun 요청/Noun 발/Noun 사이즈/Noun 몇/Noun 신으세요/Verb ', '신발 착화감 편하게/Adjective 신/Noun 을/Josa 수/Noun 있는/Adjective 거/Noun 찾으세요/Verb ', '신발 착화감 이런/Adjective 건/Noun 어떠세요/Adjective 이런/Adjective 거도/Noun 신발/Noun 무척/Noun 편하거든요/Adjective ', '신발 굽높이문의 봄/Noun 상품/Noun 은/Josa 아직/Adverb 어른/Noun 제품/Noun 이/Josa 많이/Adverb 안/VerbPrefix 나왔습니다/Verb ', '신발 재입고문의 이번/Noun 주/Noun 지나면/Verb 들어올/Verb 거/Noun 예요/Josa ']


In [41]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(cat_main_question, word_to_index, ENCODER_INPUT)

# 첫 번째 인코더 입력 출력 (신발 은 여기 있는 게 다예 요)
print(cat_main_question[0])
x_encoder[0]


신발 종류별신발제품문의요청 신발/Noun 은/Josa 여기/Noun 있는/Adjective 게/Noun 다예/Noun 요/Josa 


array([4909, 1580, 3247, 5481, 2133, 2899, 1486, 2062, 4566,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [43]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(cat_main_answer, word_to_index, DECODER_INPUT)

# 첫 번째 디코더 입력 출력 (<START> 신발 은 여기 있는 게 다예 요)
print(cat_main_answer[0])
x_decoder[0]


신발 종류별신발제품문의요청 네/Noun 성인/Noun 이나/Josa 아동/Noun 다/Adverb 있어요/Adjective 신발/Noun 종류/Noun 별/Modifier 신발/Noun 제품/Noun 문의/Noun 요청/Noun 발/Noun 사이즈/Noun 몇/Noun 신으세요/Verb 


array([   1, 4909, 1580, 5488, 1825, 5174, 3563, 4422, 1019, 3247, 2408,
         50, 3247, 5039, 4307, 4332, 6290, 4872,  972, 4448,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [44]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(cat_main_answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 입력 출력 (신발 은 여기 있는 게 다예 요 <END>)
print(cat_main_answer[0])
y_decoder[0]


신발 종류별신발제품문의요청 네/Noun 성인/Noun 이나/Josa 아동/Noun 다/Adverb 있어요/Adjective 신발/Noun 종류/Noun 별/Modifier 신발/Noun 제품/Noun 문의/Noun 요청/Noun 발/Noun 사이즈/Noun 몇/Noun 신으세요/Verb 


array([4909, 1580, 5488, 1825, 5174, 3563, 4422, 1019, 3247, 2408,   50,
       3247, 5039, 4307, 4332, 6290, 4872,  972, 4448,    2,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [ ]:
# # 원핫인코딩 초기화
# one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))

# # 디코더 목표를 원핫인코딩으로 변환
# # 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
# for i, sequence in enumerate(y_decoder):
#     for j, index in enumerate(sequence):
#         one_hot_data[i, j, index] = 1

# # 디코더 목표 설정
# y_decoder = one_hot_data

# # 첫 번째 디코더 목표 출력
# y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

인코더 입력과 디코더 입력은 임베딩 레이어에 들어가는 인덱스 배열입니다. 반면에 디코더 출력은 원핫인코딩 형식이 되어야 합니다. 디코더의 마지막 Dense 레이어에서 softmax로 나오기 때문입니다.

<br>
<br>
# 모델 생성

In [45]:
#--------------------------------------------
# 훈련 모델 인코더 정의
#--------------------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]

#--------------------------------------------
# 훈련 모델 디코더 정의
#--------------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])    

In [46]:
#--------------------------------------------
#  예측 모델 인코더 정의
#--------------------------------------------

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)

#--------------------------------------------
# 예측 모델 디코더 정의
#--------------------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

예측 모델은 이미 학습된 훈련 모델의 레이어들을 그대로 재사용합니다. 예측 모델 인코더는 훈련 모델 인코더과 동일합니다. 그러나 예측 모델 디코더는 매번 LSTM 상태값을 입력으로 받습니다. 또한 디코더의 LSTM 상태를 출력값과 같이 내보내서, 다음 번 입력에 넣습니다. 

이렇게 하는 이유는 LSTM을 딱 한번의 타임 스텝만 실행하기 때문입니다. 그래서 매번 상태값을 새로 초기화 해야 합니다. 이와 반대로 훈련할때는 문장 전체를 계속 LSTM으로 돌리기 때문에 자동으로 상태값이 전달됩니다. 

<br>
<br>
# 훈련 및 테스트

In [47]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [48]:
x_encoder[2].reshape(1, x_encoder[2].shape[0])

array([[4909, 3043, 5488, 5301, 2807, 2504, 2119, 1576, 1722, 2899, 5635,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [49]:
# 에폭 반복
for epoch in range(5):
    print('Total Epoch :', epoch + 1)

    # 훈련 시작
    history = model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=10,
                        batch_size=64,
                        verbose=1,
                        validation_split=0.1)
    # '네 봄 이니까 편하게 신 을 수 있는 거'에 대한 대답
    input_encoder = x_encoder[2].reshape(1, x_encoder[2].shape[0])
    input_decoder = x_decoder[2].reshape(1, x_decoder[2].shape[0])
    results = model.predict([input_encoder, input_decoder])
    
    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0], 1)
    
    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()

Total Epoch : 1
Epoch 1/10
103/103 [==============================] - 5s 49ms/step - loss: 2.9264 - accuracy: 0.6959 - val_loss: 1.7746 - val_accuracy: 0.7658
Epoch 2/10
103/103 [==============================] - 4s 39ms/step - loss: 1.6608 - accuracy: 0.7500 - val_loss: 1.5744 - val_accuracy: 0.7834
Epoch 3/10
103/103 [==============================] - 4s 39ms/step - loss: 1.5312 - accuracy: 0.7595 - val_loss: 1.5323 - val_accuracy: 0.7513
Epoch 4/10
103/103 [==============================] - 4s 38ms/step - loss: 1.4742 - accuracy: 0.7637 - val_loss: 1.4869 - val_accuracy: 0.7912
Epoch 5/10
103/103 [==============================] - 4s 39ms/step - loss: 1.4157 - accuracy: 0.7751 - val_loss: 1.4585 - val_accuracy: 0.7934
Epoch 6/10
103/103 [==============================] - 4s 38ms/step - loss: 1.3613 - accuracy: 0.7831 - val_loss: 1.4286 - val_accuracy: 0.7919
Epoch 7/10
103/103 [==============================] - 4s 39ms/step - loss: 1.3266 - accuracy: 0.7918 - val_loss: 1.3902 - val_

학습이 진행될수록 예측 문장이 제대로 생성되는 것을 볼 수 있습니다. 다만 여기서의 예측은 단순히 테스트를 위한 것이라, 인코더 입력과 디코더 입력 데이터가 동시에 사용됩니다. 아래 문장 생성에서는 예측 모델을 적용하기 때문에, 오직 인코더 입력 데이터만 집어 넣습니다.

<br>
<br>
# 문장 생성

In [50]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [51]:
# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
        
    return sentence

In [ ]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('이 티셔츠 사이즈 있나요')
input_seq


array([[1541,  307, 1754, 1072,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [ ]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


'네 잠시 만 기다려주세요 '

데이터셋 문장에서는 없던 '같이'라는 단어를 추가해 보았습니다. 그래도 비슷한 의미란 것을 파악하여 동일한 답변이 나왔습니다.
<br>
<br>
<br>

In [ ]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('교환 가능한가요?')
input_seq


array([[2393, 5592,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [ ]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


'네 가능합니다 '

In [52]:
# for train data predict
for seq_index in range(0,100):

  print("고객 : ",question[seq_index])
  print("정답점원 :",answer[seq_index])
  print("AI점원 :",generate_text(make_predict_input(question[seq_index])))
  print("\n")

고객 :  신발/Noun 은/Josa 여기/Noun 있는/Adjective 게/Noun 다예/Noun 요/Josa 
정답점원 : 네/Noun 성인/Noun 이나/Josa 아동/Noun 다/Adverb 있어요/Adjective 신발/Noun 종류/Noun 별/Modifier 신발/Noun 제품/Noun 문의/Noun 요청/Noun 발/Noun 사이즈/Noun 몇/Noun 신으세요/Verb 
AI점원 : 가방 종류별가방제품문의요청 네/Noun 


고객 :  230/Number 이요/Noun 
정답점원 : 편하게/Adjective 신/Noun 을/Josa 수/Noun 있는/Adjective 거/Noun 찾으세요/Verb 
AI점원 : 의류 계절상품문의 네/Noun 


고객 :  네/Noun 봄/Noun 이니까/Josa 편하게/Adjective 신/Noun 을/Josa 수/Noun 있는/Adjective 거/Noun 
정답점원 : 이런/Adjective 건/Noun 어떠세요/Adjective 이런/Adjective 거도/Noun 신발/Noun 무척/Noun 편하거든요/Adjective 
AI점원 : 의류 베스트상품문의 요즘/Noun 은/Josa 잘/Verb 나갑니다/Verb 


고객 :  굽/Noun 좀/Noun 높은/Adjective 거/Noun 없나요/Adjective 
정답점원 : 봄/Noun 상품/Noun 은/Josa 아직/Adverb 어른/Noun 제품/Noun 이/Josa 많이/Adverb 안/VerbPrefix 나왔습니다/Verb 
AI점원 : 신발 제품가격문의 네/Noun 맞습니다/Verb 


고객 :  언제/Noun 들어와요/Verb 
정답점원 : 이번/Noun 주/Noun 지나면/Verb 들어올/Verb 거/Noun 예요/Josa 
AI점원 : 액세서리 제품가격문의 네/Noun 


고객 :  이/Determiner 거/Noun 는/Josa 가죽/Noun 이에요/Josa 
정답점원 : 가죽/Noun 아니고/Adjective 쎄/Adjectiv

In [76]:
wear_data_test = pd.read_csv("/content/drive/My Drive/자연어처리,추천/NLPer/chatbot_pra/csv_file/wear_test.csv")
customer = wear_data_test[wear_data.SPEAKER == "고객"].SENTENCE

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [73]:
cat_main = list(wear_data['CATEGORY_MAIN'][14900:])
print(len(cat_main))


926


'의류 신제품,구제품관련문의'

In [78]:
question = []

for Q in customer:
    question.append(Q.replace("[^\w]", " "))

print(question[:5])

['구제는 뭐예요?', '이건 다 세탁이 된 거죠?', '구제품은 다 세탁해서 나오는 거겠죠?', '카드는 부가세 별도로 붙나요?', '카드로 하면 부가세 붙어요?']


In [79]:
question_test = pos_tag(question)

In [80]:
# for train data predict
for seq_index in range(0,100):
    input = cat_main[seq_index] + ' ' + question_test[seq_index]
    print("고객 : ",question_test[seq_index])
    print("AI점원 :",generate_text(make_predict_input(input)))
    print("\n")

고객 :  구제/Noun 는/Josa 뭐/Noun 예요/Josa 
AI점원 : 액세서리 제품문의 네/Noun 


고객 :  이건/Noun 다/Adverb 세탁/Noun 이/Josa 된/Verb 거/Noun 죠/Josa 
AI점원 : 액세서리 제품문의 네/Noun 


고객 :  구/Modifier 제품/Noun 은/Josa 다/Adverb 세탁/Noun 해서/Verb 나오는/Verb 거/Noun 겠죠/Verb 
AI점원 : 액세서리 매장에서착용문의 네/Noun 착용/Noun 해보세요/Verb 


고객 :  카드/Noun 는/Josa 부가세/Noun 별도/Noun 로/Josa 붙나요/Verb 
AI점원 : 액세서리 매장에서착용문의 네/Noun 착용/Noun 해보세요/Verb 


고객 :  카드/Noun 로/Josa 하면/Verb 부가세/Noun 붙어요/Verb 
AI점원 : 액세서리 매장에서착용문의 네/Noun 착용/Noun 해보세요/Verb 


고객 :  이/Determiner 거/Noun 적립/Noun 되죠/Verb 
AI점원 : 액세서리 매장에서착용문의 네/Noun 착용/Noun 해보세요/Verb 


고객 :  몸/Noun 에/Josa 딱/Adverb 붙는/Verb 티셔츠/Noun 있어요/Adjective 
AI점원 : 액세서리 제품문의 네/Noun 


고객 :  할인/Noun 코너/Noun 는/Josa 어디/Noun 에요/Josa 
AI점원 : 액세서리 제품문의 네/Noun 


고객 :  할인/Noun 코너/Noun 는/Josa 어느/Adverb 쪽/Noun 에/Josa 있나요/Adjective 
AI점원 : 액세서리 제품문의 네/Noun 


고객 :  영업/Noun 은/Josa 매일/Noun 하시나요/Verb 
AI점원 : 액세서리 제품문의 네/Noun 


고객 :  매일/Noun 영업/Noun 하시는/Verb 건가/Noun 요/Josa 
AI점원 : 액세서리 종류별액세서리제품문의요청 네/Noun 


고객 :  카드/Noun 가로/No